# Local Model Serving with MLflow

In [ ]:
import httpx 
import pandas as pd
import mlflow
from examples.walmart_sales_regression.data import SalesDataProcessor
from examples.utils.file_utils import get_root_dir
from examples.utils.file_utils import read_file

# set mlflow tracking uri
mlflow.set_tracking_uri(uri=(get_root_dir() / 'mlruns').as_uri())

```shell
mlflow models serve --options
```

To run the code below make sure you deploy the model using.

`poetry run mlflow models serve -m models:/walmart-store-sales-regressor@production --env-manager local`

In [ ]:
root_dir = get_root_dir()
configs = read_file(
    root_dir / "examples" / "walmart_sales_regression" / "configs.yaml"
)
registered_model_name = configs["registered_model_name"]

data_path = (
    root_dir.parents[1] / "Downloads" / "sales-walmart" / "Walmart_Sales.csv"
)  # change this to your data path

data_processor = SalesDataProcessor(path=data_path, configs=configs)

url = "http://localhost:5000/invocations"

_, x_test, _, y_test = data_processor.create_train_test_split()

# predicting for the store store_id
store_id = 3
x_test = x_test[x_test["Store"] == store_id]
y_test = y_test[y_test["Store"] == store_id]

payload = {
    "dataframe_split": x_test.to_dict(orient="split"),
    "params": {"store_id": str(store_id)},
}
headers = {"Content-Type": "application/json"}
response = httpx.post(url, json=payload, headers=headers)

In [ ]:
response.json()

In [ ]:
pd.DataFrame({
    "predictions": response.json()["predictions"],    
    "target": y_test["Weekly_Sales"].values,
}).plot()